# Online pipeline for Feldman lab

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

import spikeextractors as se
import spiketoolkit as st
import spikesorters as ss
import spikecomparison as sc
import spikewidgets as sw
import time
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

from feldman_lab_to_nwb import FeldmanNWBConverter

%matplotlib notebook

In [ ]:
# set parameters for parallelization
n_jobs = 8 # number of concurrent jobs
chunk_mb = 2000  # max amount of RAM in Mb

## 1) Load short AP and LF recordings

In [ ]:
base_path = Path("D:/Feldman/Neuropixels_Feldman/210209/SpikeGLX")
#session_name = "LR_210209_2_g1"
# session_name = "LR_210209_g1"
session_name = "LR_210209_g0"
# session_name = "LR_210209_2_g1"
ap_bin_path = base_path / session_name / f"{session_name}_imec0" / f"{session_name}_t0.imec0.ap.bin"
lf_bin_path = base_path / session_name / f"{session_name}_imec0" / f"{session_name}_t0.imec0.lf.bin"
nidq_bin_path = base_path / session_name / f"{session_name}_t0.nidq.bin"

In [ ]:
recording_ap = se.SpikeGLXRecordingExtractor(ap_bin_path)

In [ ]:
duration = recording_ap.get_num_frames() / recording_ap.get_sampling_frequency()
fs = recording_ap.get_sampling_frequency()
print(f"Duration: {np.round(duration, 1)} s")

In [ ]:
# for testing, cut out 2 minutes
subrec = se.SubRecordingExtractor(recording_ap, end_frame=120*fs)
subrec.get_num_frames() / subrec.get_sampling_frequency()

# 2) Quick spike detection by channel

In [ ]:
t_start = time.time()
sorting_ch = st.sortingcomponents.detect_spikes(
    recording=subrec,
    n_jobs=n_jobs,
    chunk_mb=chunk_mb,
    verbose=True
)
t_stop = time.time()
print(f"Elapsed time for detection: {t_stop - t_start}")

In [ ]:
print(f"Detected spikes on {len(sorting_ch.get_unit_ids())} channels")

In [ ]:
wr = sw.plot_rasters(sorting_ch)

### (optional) Remove channels below a certain firing rate

In [ ]:
firing_rate_threshold = 0.1
num_frames = subrec.get_num_frames()

sorting_high_fr = st.curation.threshold_firing_rates(
    sorting_ch,
    duration_in_frames=num_frames,
    threshold=firing_rate_threshold, 
    threshold_sign="less"
)

In [ ]:
print(f"Detected spikes on {len(sorting_high_fr.get_unit_ids())} channels with fr > {firing_rate_threshold}")

# 3) Save spike and behavior info to NWB

In [ ]:
# Specify behavioral folder path and name of NWBFile
behavior_folder_path = "D:/Feldman/ADRIAN"
nwbfile_path = f"D:/Feldman/rapid_testing_{session_name}.nwb"

# Choose a sorting extractor (either basic detection or rate thresholded)
chosen_sorting = sorting_high_fr


# Run conversion to NWB
source_data = dict(Behavior=dict(folder_path=behavior_folder_path))
conversion_options = dict(Behavior=dict(nidq_synch_file=str(nidq_bin_path)))
converter = FeldmanNWBConverter(source_data=source_data)
metadata = converter.get_metadata()
metadata["NWBFile"].update(session_description="Rapid testing file for electrode placement.")
converter.run_conversion(
    nwbfile_path=nwbfile_path,
    metadata=metadata,
    conversion_options=conversion_options,
    overwrite=True  # this creates the file
)

pprint("Appending recording and sorting info...")
se.NwbRecordingExtractor.write_recording(
    recording=subrec,
    save_path=nwbfile_path,
    overwrite=False  # this appends the file. True would write a new file
)
se.NwbSortingExtractor.write_sorting(
    sorting=chosen_sorting,
    save_path=nwbfile_path,
    overwrite=False  # this appends the file. True would write a new file
)
pprint("Recording and sorting appended!")

# 5) View output vs. behavior in NWBWidgets 

In [ ]:
io = NWBHDF5IO(nwbfile_path, mode='r')
nwb = io.read()

nwb2widget(nwb)

In [ ]:
io.close()